In [ ]:
import pandas as pd
import nibabel as nib
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sys
sys.path.append('..')
import sfp
import pyPyrTools as ppt
import math
from scipy import stats
from scipy import optimize as opt
import torch

To explain the motivation behind this model, let's step through some reasoning.

In [ ]:
# Some setup for torch, loading in data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
sns.set_style('whitegrid')

df_path = "/scratch/wfb229/spatial_frequency_preferences/derivatives/first_level_analysis/stim_class/posterior/sub-wlsubj045/ses-02/sub-wlsubj045_ses-02_task-sfp_v1_e1-12_summary.csv"
df_path = "/home/billbrod/Data/sub-wlsubj045_ses-02_task-sfp_v1_e1-12_summary.csv"
df = pd.read_csv(df_path)

In [ ]:
# This is necessary for now, so we normalize values across voxels, but eventually we'll take care of this during the first level analysis
gb = df.groupby(['varea', 'voxel'])
df = df.set_index(['varea', 'voxel'])
df['amplitude_estimate_norm'] = gb.amplitude_estimate_median.apply(np.linalg.norm, 2)
df = df.reset_index()
df['amplitude_estimate_median_normed'] = df.amplitude_estimate_median / df.amplitude_estimate_norm

Let's look at some voxels that have good GLM $R^2$ values and pick one

In [ ]:
df[(df.varea==1)&(df.R2>55)].drop_duplicates('voxel').sort_values('R2', ascending=False).head(20)[['voxel', 'R2', 'varea', 'hemi', 'angle' ,'eccen', 'precision']]

In [ ]:
# Pick a V1 voxel with a good R2
voxel_df = df[(df.voxel.isin([264]))]#, 1421]))]
voxel_df.head()

In [ ]:
sns.set_context('poster')
sns.set_style('white')

In [ ]:
pal = sns.palettes.color_palette('deep', 5)
pal = {'radial': pal[0], 'reverse spiral': pal[4], 'forward spiral': pal[2], 'angular': pal[3], 'mixtures': pal[1]}
hue_order = ['radial', 'reverse spiral', 'forward spiral', 'angular', 'mixtures']

Let's examine the response of this voxel as a function of spatial frequency. In the plot below, we plot the normed amplitude estimate as a function of the local spatial frequency. We see that the response looks roughly log-Normal, but there appears to be some difference between the different stimulus classes.

In [ ]:
g = sns.FacetGrid(voxel_df[~voxel_df.stimulus_superclass.isin(['mixtures'])], hue='stimulus_superclass',palette=pal, size=8, aspect=1.5, hue_order=hue_order)
g.map(plt.scatter, 'local_sf_magnitude', 'amplitude_estimate_median', linewidth=6)
g.map(plt.plot, 'local_sf_magnitude', 'amplitude_estimate_median', linewidth=6)
# g.ax.set_xscale('log', basex=2)
g.add_legend()
g.ax.tick_params(size=0)
g.ax.set_xlim((0, 6))
g.set_xlabels('Local spatial frequency (cpd)')
g.set_ylabels('Amplitude Estimate')
g.fig.savefig('voxel.svg')

These classes differ in their local orientation, so we can look at a plot of the response as a function of the local spatial frequency with respect to x and y (size represents the response). This plot is difficult to parse, but the main point is that these different stimulus classes are not arbitrary and discrete: they lie on a continuum, related by the stimulus orientation, and so we can fit the response of the voxel as a 2d tuning curve.

In [ ]:
def scatter_sizes(x, y, s, plot_color=False, cmap=None, size_scale=1, **kwargs):
    if plot_color:
        kwargs.pop('color')
        if cmap is None:
            cmap = 'Blues'
        plt.scatter(x, y, s=s*80*size_scale, c=s, cmap=cmap, **kwargs)
    else:
        plt.scatter(x, y, s=s*80*size_scale, **kwargs)

with sns.axes_style('whitegrid'):
    voxel_df['normalized_resp'] = voxel_df['amplitude_estimate_median'].copy()
    voxel_df['normalized_resp'] = (voxel_df['normalized_resp'] - voxel_df['normalized_resp'].min()) / (voxel_df['normalized_resp'].max() - voxel_df['normalized_resp'].min())
    g=sns.FacetGrid(voxel_df, size=8, aspect=1, hue='stimulus_superclass', palette=pal, hue_order=hue_order)
    g.map(scatter_sizes, 'local_w_x', 'local_w_y', 'normalized_resp', plot_color=False, size_scale=2)
    g.add_legend()
    scatter_ax = plt.gca()
    scatter_ax.set_aspect('equal')
    g.ax.tick_params(size=0)
    g.fig.savefig('voxel-polar.svg')

This is just that same data, but rotated and plotted on a semi-log plots.

In [ ]:
with sns.axes_style('whitegrid'), sns.plotting_context('notebook'):
    g=sns.FacetGrid(voxel_df, hue='stimulus_superclass', size=5, aspect=1, palette=pal, hue_order=hue_order)
    g.map(scatter_sizes, 'local_w_r', 'local_w_a', 'normalized_resp')
    g.add_legend()
    scatter_ax = plt.gca()
    scatter_ax.set_xscale('symlog', basex=2, linthreshx=2**(-4))
    scatter_ax.set_yscale('symlog', basey=2, linthreshy=2**(-4))

But then the question is: how does the tuning change with orientation? Two possibilities are:

1. The preferred frequency of the tuning curve / mode of the log-Gaussian distribution changes with orientation.
2. The amplitude of the tuning curve changes with orientation.

Then there's the question of how either the mode or the amplitude changes with orientation. Let's assume it changes smoothly and periodically, symmetrically about 180 degrees (because 2d orientation is runs from 0 to 180 degrees). We'll examine three possibilities in the plots below, from left to right:

1. all orientation are equally important (mode/amplitude does not depend on orientation; constant)
2. horizontal or vertical is preferred, but the other is anti-preferred (sinusoid with frequency $2\theta$)
3. the cardinals are preferred, the obliques are anti-preferred (sinusoid with frequency $4\theta$)

The following plots show these three possibilities, with the top showing a plot of orientation vs mode/response, while the bottom shows this on an x/y plot: the line represents either the mode or the level set of the max amplitude.

In [ ]:
with sns.plotting_context('paper'), sns.axes_style('white'):
    x = np.linspace(.1, 30, 100)
    y = sfp.tuning_curves.log_norm_pdf(x,1, 6, .45)
    plt.plot(x, y, 'k')
    plt.savefig('loggauss2.svg')

In [ ]:
with sns.plotting_context('paper'):
    fig = plt.figure(figsize=(12,6))
    theta = np.linspace(0, np.pi, 1000)
    for i, f, title in zip(range(3), [0, 2, 4], ['1. constant', '2. horizontal', '3. cardinals']):
        r = (np.cos(f*theta) + 3) / 4
        ax = plt.subplot(2,3,i+1)
        ax.plot(theta, r)
        ax.tick_params(size=0)
        sns.despine(ax=ax)
        ax.set_title(title)
        ax = plt.subplot(2,3,4+i, projection='polar')
        ax.plot(theta, r)
    fig.savefig('variations.svg')

Okay so then how do we set up the function that we're going to fit? First, let's look at the 1d log-Normal distribution we used before. Normally, this distribution is parameterized by $\mu$ and $\sigma^2$. These are *not* the mean and variance, the way they are for the regular Normal distribution. We'll keep using $\sigma^2$ but we'll use the mode, $M$ instead of $\mu$: $M = \exp(\mu - \sigma^2) \Rightarrow \mu = \ln(M) + \sigma^2$.

Thus, when we're modeling the response, $R$ as a 1d log-Normal tuning curve with respect to the spatial frequency $\omega$, it's: $R=A * \exp(-\frac{(\ln(\omega)-\ln(M)-\sigma^2)^2}{2\sigma^2})$.

Now we're extending this to make it 2d, as a funtion of spatial frequency $\omega$ and orientation $\theta$. To allow for the amplitude varying with orientation, we expand upon $A$ and make it orientation-dependent: $A_0 + A_1\cos2\theta + A_2\cos4\theta$. To allow the mode to vary, we make similarly make $M$ orientation-dependent: $M_0 + M_1\cos2\theta + M_2\cos4\theta$.

Putting it altogether we get:

$(A_0 + A_1\cos2\theta + A_2\cos4\theta)\exp(-\frac{(\ln(\omega)-\ln(M_0 + M_1\cos2\theta + M_2\cos4\theta)-\sigma^2)^2}{2\sigma^2})$

which gives us 7 parameters to fit: $A_0, A_1, A_2, M_0, M_1, M_2, \sigma$

Let's look at some examples of this function, to get a sense of its expressive power.

In [ ]:
omega = np.logspace(-3, 3, 100, base=2)
theta = np.linspace(0, 2*np.pi, 100)

def log_norm_2d(omega, theta, A0=1, A1=0, A2=0, M0=1, M1=0, M2=0, sigma=1):
    omega = np.array(omega)
    theta = np.array(theta)
    amp = A0 + A1*np.cos(2*theta) + A2*np.cos(4*theta)
    mode = M0 + M1*np.cos(2*theta) + M2*np.cos(4*theta)
    mu = np.log(mode) + sigma**2
    pdf = (1/(omega*sigma*np.sqrt(2*np.pi))) * np.exp(-(np.log(omega)-mu)**2/(2*sigma**2))
    #pdf /= pdf.max()
    return amp * pdf

omega, theta = np.meshgrid(omega, theta)

In [ ]:
fig, axes = plt.subplots(2, 3, subplot_kw={'projection': 'polar'}, figsize=(12, 6))

params = [{}, {'M0': .5}, {'A1': .5}, {'M1': .5}, {'A1': .5, 'M1': .5, 'M0': 1}, {'A1': .5, 'M1': .5, 'M0':2}]

with sns.plotting_context('notebook'):
    for i, ax in enumerate(axes.flatten()):
        R = log_norm_2d(omega, theta, **params[i])
        ax.pcolormesh(theta, omega, R)
        ax.set_xticks([])
        ax.set_rticks([])
    fig.savefig('2d.png')

This is on a voxel-by-voxel basis, but what about across the whole area? To extend it to all of V1, let's consider two frames of reference: fixed and relative.

In the fixed frame, all voxels have the same tuning. Orientation, $\theta_f$, above refers to Cartesian, world-relate orientation so that $\theta_f=0$ corresponds to "to the right". Spatial frequency, $\omega_f$, means the local spatial frequency in the image. This encodes our "constant" extreme possibility from earlier.

In the relative frame, voxel tuning depends on its location in the retinotopic map. We remap orientation and spatial frequency so that $\theta_r=0$ corresponds to "away from the fovea" and spatial frequency is scaled by eccentricity: $\omega_r=\omega_f(e+b)$, where $e$ is the eccentricity of the voxel's pRF and $b$ is some constant.

We then sum these two versions of the model so that we have a 15-parameter model (two versions of the 7 parameters above, plus $b$) that we fit simultaneously to all of V1.

# Torch

In [ ]:
import warnings
def torch_meshgrid(x, y=None):
    """from https://github.com/pytorch/pytorch/issues/7580"""
    if y is None:
        y = x
    x = torch.tensor(x, dtype=torch.float64)
    y = torch.tensor(y, dtype=torch.float64)
    m, n = x.size(0), y.size(0)
    grid_x = x[None].expand(n, m)
    grid_y = y[:, None].expand(n, m)
    return grid_x, grid_y

def _cast_as_param(x):
    return torch.nn.Parameter(torch.tensor(x, dtype=torch.float64))

class LogGaussianDonut(torch.nn.Module):
    """LogGaussianDonut in pytorch
    """
    def __init__(self, major_axis_slope, minor_axis_slope, major_axis_sigma_slope, minor_axis_sigma_slope, rotation_angle_sf_coeff=0, rotation_angle_vox_coeff=0, major_axis_intercept=0, minor_axis_intercept=0, major_axis_sigma_intercept=0, minor_axis_sigma_intercept=0, rotation_angle_intercept=0):
        super(LogGaussianDonut, self).__init__()
        self.amplitude_dict = {}
        self.major_axis_slope = _cast_as_param(major_axis_slope)
        self.major_axis_intercept = _cast_as_param(major_axis_intercept)
        self.minor_axis_slope = _cast_as_param(minor_axis_slope)
        self.minor_axis_intercept = _cast_as_param(minor_axis_intercept)
        self.major_axis_sigma_slope = _cast_as_param(major_axis_sigma_slope)
        self.major_axis_sigma_intercept = _cast_as_param(major_axis_sigma_intercept)
        self.minor_axis_sigma_slope = _cast_as_param(minor_axis_sigma_slope)
        self.minor_axis_sigma_intercept = _cast_as_param(minor_axis_sigma_intercept)
        self.rotation_angle_sf_coeff = _cast_as_param(rotation_angle_sf_coeff)
        self.rotation_angle_vox_coeff = _cast_as_param(rotation_angle_vox_coeff)
        self.rotation_angle_intercept = _cast_as_param(rotation_angle_intercept)
    
    def initialize_amplitude_dict(self, amp_dict):
        amp_dict = list(amp_dict.iteritems())
        self.amplitude_dict = dict((("%.05f"%k[0], "%.05f"%k[1]), i) for i, (k, _) in enumerate(amp_dict))
        self.amplitude_list = torch.nn.ParameterList([_cast_as_param([v]) for _, v in amp_dict])
    
    def _create_mag_angle(self, extent=(-10, 10), n_samps=1001):
        x = torch.linspace(extent[0], extent[1], n_samps)
        x, y = torch_meshgrid(x)
        r = torch.sqrt(torch.pow(x, 2) + torch.pow(y, 2))
        th = torch.atan2(y, x)
        return r, th
    
    def create_image(self, vox_ecc, vox_angle, extent=None, n_samps=1001):
        r, th = self._create_mag_angle(extent, n_samps)
        return self.evaluate(r, th, vox_ecc, vox_angle)

    def get_func_params(self, voxel_eccentricity, voxel_angle):
        amplitude = []
        try:
            ecc = [i for i in voxel_eccentricity.cpu().numpy()]
            ang = [i for i in voxel_angle.cpu().numpy()]
        except TypeError:
            ecc = [float(voxel_eccentricity.cpu().numpy())]
            ang = [float(voxel_angle.cpu().numpy())]
        for e, a in zip(ecc, ang):
            amp_idx = self.amplitude_dict[("%.05f"%e, "%.05f"%a)]
            amplitude.append(self.amplitude_list[amp_idx])
        amplitude = torch.cat(amplitude)
        major_axis = 1. / (self.major_axis_slope * voxel_eccentricity + self.major_axis_intercept)
        minor_axis = 1. / (self.minor_axis_slope * voxel_eccentricity + self.minor_axis_intercept)
        major_axis_sigma = 1. / (self.major_axis_sigma_slope * voxel_eccentricity + self.major_axis_sigma_intercept)
        minor_axis_sigma = 1. / (self.minor_axis_sigma_slope * voxel_eccentricity + self.minor_axis_sigma_intercept)
        rotation_angle_func = lambda sf_angle: self.rotation_angle_sf_coeff * sf_angle + self.rotation_angle_vox_coeff * voxel_angle + self.rotation_angle_intercept
        return amplitude, major_axis, minor_axis, major_axis_sigma, minor_axis_sigma, rotation_angle_func
    
    def evaluate(self, sf_mag, sf_angle, vox_ecc, vox_angle):
        variables = {'sf_mag': sf_mag, 'sf_angle': sf_angle, 'vox_ecc': vox_ecc, 'vox_angle': vox_angle}
        for k, v in variables.iteritems():
            if not torch.is_tensor(v):
                v = torch.tensor(v, dtype=torch.float64)
            if self.major_axis_slope.is_cuda:
                v = v.cuda()
            variables[k] = v
        amplitude, major_axis, minor_axis, major_axis_sigma, minor_axis_sigma, rotation_angle_func = self.get_func_params(variables['vox_ecc'], variables['vox_angle'])
        rotation_angle = rotation_angle_func(variables['sf_angle'])
        variables['sf_mag'] = torch.log2(variables['sf_mag'])
        variables['sf_angle'] = variables['sf_angle'] + rotation_angle
        # transform angles based on ellipse axes
        variables['sf_angle'] = torch.atan2(major_axis*torch.sin(variables['sf_angle']), minor_axis*torch.cos(variables['sf_angle']))
        # Gaussian center as function of angle
        self.ctr = torch.sqrt(torch.pow(major_axis*torch.cos(variables['sf_angle']), 2) + torch.pow(minor_axis*torch.sin(variables['sf_angle']), 2))
        # rotational sigma
        self.sigma = torch.sqrt(torch.pow(major_axis_sigma*torch.cos(variables['sf_angle']), 2) + torch.pow(minor_axis_sigma*torch.sin(variables['sf_angle']),2))
        # This is our function
        return amplitude*torch.exp(-(variables['sf_mag']-torch.log2(self.ctr))**2 / (2*self.sigma**2))

    def forward(self, spatial_frequency_magnitude, spatial_frequency_theta, voxel_eccentricity, voxel_angle):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.evaluate(spatial_frequency_magnitude, spatial_frequency_theta, voxel_eccentricity, voxel_angle)
    
    
class ConstantLogGaussianDonut(LogGaussianDonut):
    """this version does not depend on voxel eccentricity or angle at all, mainly for demo purposes"""
    def __init__(self, amplitude, major_axis, minor_axis, major_axis_sigma, minor_axis_sigma, rotation_angle):
        # This way, the {major|minor}_axis(_sigma) is what the user specifies (since we takes the inverse above)
        super(ConstantLogGaussianDonut, self).__init__(0, 0, 0, 0, 0, 0, 1./major_axis, 1./minor_axis, 1./major_axis_sigma, 1./minor_axis_sigma, rotation_angle)
        self.initialize_amplitude_dict({(0, 0): amplitude})
        
    def create_image(self, extent=None, n_samps=1001):
        r, th = self._create_mag_angle(extent, n_samps)
        return self.evaluate(r, th)
        
    def evaluate(self, sf_mag, sf_angle):
        return super(ConstantLogGaussianDonut, self).evaluate(sf_mag, sf_angle, 0, 0)
        
    def forward(self, spatial_frequency_magnitude, spatial_frequency_theta):
        return self.evaluate(spatial_frequency_magnitude, spatial_frequency_theta)

In [ ]:
donut = ConstantLogGaussianDonut(20, 1, 1, .5, .5,0).to(device)
x = np.linspace(-5, 5, 1001)
xgrid, ygrid = np.meshgrid(x, x)
# detach() is required to separate it from the graph implied by setting `requires_grad=True` above
plt.imshow(donut.create_image((x.min(), x.max())).detach(), extent=(x.min(),x.max(), x.min(), x.max()), cmap='Reds')
plt.colorbar()

In [ ]:
donut = LogGaussianDonut(1, 1, 1./.5, 1/.5, 0)
donut.initialize_amplitude_dict({(1, 0): 20})
donut.to(device)
x = np.linspace(-5, 5, 1001)
xgrid, ygrid = np.meshgrid(x, x)
# detach() is required to separate it from the graph implied by setting `requires_grad=True` above
plt.imshow(donut.create_image(1, 0, (x.min(), x.max())).detach(), extent=(x.min(),x.max(), x.min(), x.max()), cmap='Reds')
plt.colorbar()

In [ ]:
img = donut.create_image(1, 0, (x.min(), x.max()), len(x)).cpu().detach().numpy()
fig, axes = plt.subplots(2,2,figsize=(10, 5))
axes=axes.flatten()
R = ppt.mkR(len(x))
R *= (np.sqrt(2*x.max()**2)/R.max())
R[xgrid<0] *= -1
for ax, a in zip(axes.flatten(), [0, 1, 2, 3]):
    idx = np.where(xgrid==a*ygrid)
    r = R[idx]
    ax.plot(r, img[idx])
    ax.set(xlim=(-8, 8))

In [ ]:
def create_amplitude_init_dict(df):
    df = df.groupby('voxel')[['amplitude_estimate_median', 'eccen', 'angle']].max()
    df = df.reset_index().set_index(['eccen', 'angle'])
    amp_dict = {}
    for idx, row in df.iterrows():
        amp_dict[idx] = row.amplitude_estimate_median
    return amp_dict

In [ ]:
ds = FirstLevelDataset(df_path)

In [ ]:
g=sns.FacetGrid(voxel_df, size=5, aspect=1)
g.map(scatter_sizes, 'local_w_x', 'local_w_y', 'normalized_resp')
scatter_ax = plt.gca()
scatter_ax.set_aspect('equal')

mag = torch.tensor(voxel_df.local_sf_magnitude.values, dtype=torch.float64).to(device)
direc = torch.tensor(voxel_df.local_sf_xy_direction.values, dtype=torch.float64).to(device)
ecc = torch.tensor(voxel_df.eccen.values, dtype=torch.float64).to(device)
ang = torch.tensor(voxel_df.angle.values, dtype=torch.float64).to(device)
y = torch.tensor(voxel_df.amplitude_estimate_median.values, dtype=torch.float64).to(device)

donut = LogGaussianDonut(1, 1, .2, .2)
donut.initialize_amplitude_dict(create_amplitude_init_dict(ds.df))
donut.to(device)
x = np.linspace(-3, 3, 101)
# detach() is required to separate it from the graph implied by setting `requires_grad=True` above
c = scatter_ax.contour(x, x, donut.create_image(ecc[0], ang[0], (x.min(), x.max()), len(x)).detach(), cmap="Reds")
g.fig.colorbar(c, shrink=.5)
scatter_ax.set(xlim=(-4.5, 4.5), ylim=(-.5,5))
scatter_ax.set_aspect('equal')


loss_fn = torch.nn.MSELoss(False)
optimizer = torch.optim.Adam(donut.parameters(), lr=1e-3)

In [ ]:
list(donut.named_parameters())

In [ ]:
from torch.utils import data as torchdata
class FirstLevelDataset(torchdata.Dataset):
    def __init__(self, df_path, direction_type='relative'):
        self.df = pd.read_csv(df_path)
        if direction_type not in ['relative', 'absolute']:
            raise Exception("Don't know how to handle direction_type %s!" % direction_type)
        self.direction_type = direction_type
        
    def __getitem__(self, idx):
        row = df.iloc[idx]
        if self.direction_type == 'relative':
            vals = row[['local_sf_magnitude', 'local_sf_ra_direction', 'eccen', 'angle']].values
        elif self.direction_type == 'absolute':
            vals = row[['local_sf_magnitude', 'local_sf_xy_direction', 'eccen', 'angle']].values
        feature = torch.tensor(vals.astype(float), dtype=torch.float64)
        try:
            target = torch.tensor(row['amplitude_estimate'], dtype=torch.float64)
        except KeyError:
            target = torch.tensor(row['amplitude_estimate_median'], dtype=torch.float64)
        return feature, target
            
    def __len__(self):
        return self.df.shape[0]

In [ ]:
dl = torchdata.DataLoader(ds, 50)
loss_prev = 0.01
n_epochs = 5
thresh = .00001
for t in range(n_epochs):
    for i, (features, target) in enumerate(dl):
        predictions = donut(*features.transpose(1, 0))
        loss = loss_fn(predictions, target.to(device))
        if i % 100 == 0:
            print(i, loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if abs((loss - loss_prev) / loss_prev) < thresh:
            break
        loss_prev = loss
print("Final loss: %02f" % loss)

In [ ]:
list(donut.named_parameters())

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20,5))
vals = [y.cpu().detach().numpy(), y_pred.cpu().detach().numpy(), y.cpu().detach().numpy() - y_pred.cpu().detach().numpy()]
titles = ['ground truth', 'predicted', 'ground truth - predicted']
for ax, v, t in zip(axes.flatten(), vals, titles):
    scaled_v = (v - abs(v).min()) / (abs(v).max() - abs(v).min())
    pts=ax.scatter(voxel_df['local_w_x'], voxel_df['local_w_y'], s=abs(scaled_v)*50, c=v, cmap='RdBu_r', norm=sfp.plotting.MidpointNormalize(midpoint=0))
    ax.set_aspect('equal')
    plt.colorbar(pts, ax=ax, shrink=.6)
    ax.set(xlim=(-4.5, 4.5), ylim=(-.5, 5))
    ax.set_title(t)

In [ ]:
x = np.linspace(-8, 8, 1001)
# detach() is required to separate it from the graph implied by setting `requires_grad=True` above
plt.imshow(donut.create_image((x.min(), x.max())).cpu().detach(), extent=(x.min(),x.max(), x.min(), x.max()),cmap='RdBu_r', norm=sfp.plotting.MidpointNormalize(midpoint=0))
ax = plt.gca()
ax.set(xlim=(-4.5, 4.5), ylim=(-.5, 5))
plt.colorbar(shrink=.7)